## UW Pharmacy Student Self-Care Analysis (spring 3_4)

This notebook is a draft using the "spring 3_4" page in the student comment data of the 2022-2023 SY in the UW SoP. The analysis roughly looks at the determinants of the categories of self-care students chose for this quarter using VADER and NLTK sentiment analysis and Pandas table manipulation.

Note: this notebook relies on uploading a single csv sheet for one quarter, along with adding two additional columns called "Category 1" and "Category 2" which categorize each comment based on the 8 facets of self-care.

### Setup

In [1]:
#import python libraries
import numpy as np
import pandas as pd

!pip install vaderSentiment
!pip install --upgrade pip
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

from textblob import TextBlob
import nltk
nltk.download('brown')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

import spacy

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
/shared-libs/python3.9/py/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-06-01 17:40:53.025430: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropria

In [2]:
# Can change the file csv right here
file_csv = "spring3_4.csv"

data = pd.read_csv(file_csv)

first_column = data.columns[1]
data = data[[first_column, "Category 1", "Category 2"]]

In [3]:
all_dups = pd.DataFrame()
for i in range(len(data)):
    if not pd.isnull(data["Category 2"][i]):
        temp = data[[first_column, "Category 2"]].iloc[i]
        all_dups = all_dups.append(temp)
    else:
        continue
all_dups = all_dups[[first_column, "Category 2"]]
all_dups = all_dups.rename(columns={"Category 2": "Category"})

### Working data

In [4]:
data = data.rename(columns={"Category 1": "Category"})
data = data[[first_column, "Category"]]
data = data.append(all_dups)
data

,Spr 3_4,Category
0,Spent some time on the phone catching up with ...,Community
1,"For my self care, I decided to find new music ...",Emotional
2,I took the time to reach out and talk to one o...,Community
3,I took a walk around my neighborhood while lis...,Environmental
4,I am increasing my cardio work out in the apar...,Physical
...,...,...
100,I went to Starbucks for coffee and planned an ...,Community
102,I went gyming with my friends,Community
103,I'm going grocery shopping after class today! ...,Physical
105,This week for health and wellness I worked on ...,Mental


### Category frequencies within student comments

The comments data is displayed here in terms of each of the 8 self-care categories (physical, mental, community, emotional, environmental, spiritual, and occupational) through a pie chart. 

In [5]:
import plotly.express as px

dfg = data.groupby("Category").count().sort_values(by=first_column, ascending=False)
print("All reflections processed (includes duplicate categories): ", np.sum(dfg[first_column]))

dfg_pie = px.pie(dfg.reset_index(), values='Spr 3_4', names='Category', title='Category Frequencies')
dfg_pie

All reflections processed (includes duplicate categories):  154


### Sentiment analysis (positivity polarity score for each comment)

In this section, I scored each of the comments using the VADER sentiment analysis function. The function sentiment_scores(sentence) will return the positivity polarity score for each comment probabilistically between -1 and 1, with -1 being the most negative and 1 being the most positive. Note that this model is untrained and will be a very rough interpretation of the comments, i.e. not trained on what self-care is as a whole but on the general tone of each comment.

In [6]:
# function to print sentiments
# of the sentence (vader)

#taken from geekforgeeks
def sentiment_scores(sentence):
 
    # Create a SentimentIntensityAnalyzer object.
    sid_obj = SentimentIntensityAnalyzer()
 
    # polarity_scores method of SentimentIntensityAnalyzer
    # object gives a sentiment dictionary.
    # which contains pos, neg, neu, and compound scores.
    sentiment_dict = sid_obj.polarity_scores(sentence)

    return sentiment_dict['compound']

In [7]:
scores = pd.DataFrame(columns={"Win 1_2", "Score"})
for i in data[first_column]:
    sentence = str(i)
    score = sentiment_scores(sentence)
    scores = scores.append({first_column: sentence, "Score": score}, ignore_index=True)

In [8]:
scores_categories = data.merge(scores, on=first_column, how="right")
scores_categories

,Spr 3_4,Category,Score,Win 1_2
0,Spent some time on the phone catching up with ...,Community,0.7345,NaN
1,"For my self care, I decided to find new music ...",Emotional,0.9039,NaN
2,I took the time to reach out and talk to one o...,Community,0.7184,NaN
3,I took a walk around my neighborhood while lis...,Environmental,0.0000,NaN
4,I took a walk around my neighborhood while lis...,Emotional,0.0000,NaN
...,...,...,...,...
247,I'm going grocery shopping after class today! ...,Physical,0.4003,NaN
248,This week for health and wellness I worked on ...,Occupational,0.9313,NaN
249,This week for health and wellness I worked on ...,Mental,0.9313,NaN
250,Taking time this morning to sit and relax. I a...,Mental,0.7269,NaN


In [9]:
fig = px.scatter(scores_categories, y="Score", x="Category", title="Polarity Scores by Category")
mean_scores = scores_categories.groupby('Category').mean()
for c in mean_scores.index:
    fig.add_scatter(x=[c],
                    y=[mean_scores.loc[c]['Score']],
                    marker=dict(
                        color='red',
                        size=10
                    ),
                name=f'{c} mean')

fig.show()

In [10]:
px.bar(mean_scores.reset_index().sort_values(by="Score", ascending=False), x="Category", y="Score", title="Average Scores per Category")

### Scores of each category 

In [11]:
print('Physical category table')
scores_categories_physical = scores_categories[scores_categories["Category"] == "Physical"]
scores_categories_physical

Physical category table


,Spr 3_4,Category,Score,Win 1_2
5,I am increasing my cardio work out in the apar...,Physical,0.8999,NaN
7,Because I didn't have to drive this morning to...,Physical,0.7264,NaN
10,Took a nap,Physical,0.0000,NaN
11,"After a good conversation about naps, I rememb...",Physical,0.7263,NaN
17,This week I was able to go on a hike with my d...,Physical,0.0000,NaN
...,...,...,...,...
236,I went to the gym!,Physical,0.0000,NaN
238,So I have been quite sick these past few days ...,Physical,0.0653,NaN
242,I went to Starbucks for coffee and planned an ...,Physical,0.0000,NaN
244,I went gyming with my friends,Physical,0.4767,NaN


In [12]:
print('Mental category table')
scores_categories_mental = scores_categories[scores_categories["Category"] == "Mental"]
scores_categories_mental

Mental category table


,Spr 3_4,Category,Score,Win 1_2
15,This morning I bought myself some flowers to d...,Mental,0.8805,NaN
26,I was feeling frustrated about the upcoming te...,Mental,0.9127,NaN
33,Actually just went shopping for supplies for m...,Mental,0.0000,NaN
44,I have finished a book I started in the summer.,Mental,0.0000,NaN
46,I have been redecorating my room and bought ne...,Mental,0.4404,NaN
49,I watched my favorite movie.,Mental,0.4588,NaN
57,I finished a small doodle.,Mental,0.0000,NaN
59,Folded my MOUNTAIN of laundry. Still laundry o...,Mental,0.0000,NaN
72,I played the piano and worked on old songs tha...,Mental,0.6249,NaN
75,I bought myself pretty tulips from Trader Joe’s,Mental,0.4939,NaN


In [13]:
print('Community category table')
scores_categories_community = scores_categories[scores_categories["Category"] == "Community"]
scores_categories_community

Community category table


,Spr 3_4,Category,Score,Win 1_2
0,Spent some time on the phone catching up with ...,Community,0.7345,NaN
2,I took the time to reach out and talk to one o...,Community,0.7184,NaN
6,I chose to practice reset and relax. This week...,Community,0.9509,NaN
14,I took a walk outside. A long one. Up most of ...,Community,0.8573,NaN
22,I love listening to music so I have been shari...,Community,0.8070,NaN
...,...,...,...,...
234,Went to dinner with other classmates to break ...,Community,0.2732,NaN
241,I form a study group with my friends for medic...,Community,0.4767,NaN
243,I went to Starbucks for coffee and planned an ...,Community,0.0000,NaN
245,I went gyming with my friends,Community,0.4767,NaN


In [14]:
print('Emotional category table')
scores_categories_emotional = scores_categories[scores_categories["Category"] == "Emotional"]
scores_categories_emotional

Emotional category table


,Spr 3_4,Category,Score,Win 1_2
1,"For my self care, I decided to find new music ...",Emotional,0.9039,NaN
4,I took a walk around my neighborhood while lis...,Emotional,0.0000,NaN
21,I love listening to music so I have been shari...,Emotional,0.8070,NaN
34,"During our cohort meetings, my group talked ab...",Emotional,0.9620,NaN
36,"For my health and wellness for this week, I ch...",Emotional,-0.1531,NaN
42,I want to spend this moment to make a list of ...,Emotional,0.9719,NaN
54,Took the opportunity today to listen to bird s...,Emotional,0.9080,NaN
109,I bought my first pair of wireless earbuds to ...,Emotional,0.0000,NaN
150,I spent 20 minutes journaling my frustrations ...,Emotional,-0.4588,NaN
157,I took a walk around my neighborhood while lis...,Emotional,0.0000,NaN


In [15]:
print('Spiritual category table')
scores_categories_spiritual = scores_categories[scores_categories["Category"] == "Spiritual"]
scores_categories_spiritual

Spiritual category table


,Spr 3_4,Category,Score,Win 1_2
79,This week I spent half an hour meditating with...,Spiritual,0.4201,NaN
85,"This week is the holiday of Passover, so I was...",Spiritual,0.9571,NaN
127,I practiced mindfulness mediation.,Spiritual,0.0000,NaN
145,This week Egyptians are having an intense week...,Spiritual,0.4019,NaN
198,This week I spent half an hour meditating with...,Spiritual,0.4201,NaN
200,"This week is the holiday of Passover, so I was...",Spiritual,0.9571,NaN


In [16]:
print('Occupational category table')
scores_categories_occupational = scores_categories[scores_categories["Category"] == "Occupational"]
scores_categories_occupational

Occupational category table


,Spr 3_4,Category,Score,Win 1_2
20,"Yesterday, I took a break from studying to try...",Occupational,0.5709,NaN
53,After WIP I came home and spent 20 minutes to ...,Occupational,0.0000,NaN
63,I met my friends over the weekend for a meal t...,Occupational,0.4767,NaN
84,I used class time to finish watching my vaccin...,Occupational,0.0000,NaN
130,I joined a study group with my friends to prep...,Occupational,0.4767,NaN
138,So I have been quite sick these past few days ...,Occupational,0.0653,NaN
140,I form a study group with my friends for medic...,Occupational,0.4767,NaN
151,This week for health and wellness I worked on ...,Occupational,0.9313,NaN
165,"Yesterday, I took a break from studying to try...",Occupational,0.5709,NaN
185,After WIP I came home and spent 20 minutes to ...,Occupational,0.0000,NaN


### Frequency of nouns and verbs 

In [17]:
!pip install spacy -q
!python -m spacy download en_core_web_sm -q

from collections import Counter
import en_core_web_sm

2023-06-01 17:41:11.528463: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-01 17:41:11.715158: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-06-01 17:41:11.715204: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-06-01 17:41:11.746931: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-06-01 17:41:12.637084: W tensorflow/stream_executor/pla

Note: Just for the sake of experimentation, I only did this part on comments categorized as "physical." Can easily broaden it to more categories if useful.

This section takes the top 100 ('top' meaning most frequent) words from a particular category along with its count within the comments. Again, note that there may be duplicates depending on if the comment was repeated if it was categorized twice. I then created a function to extract nouns and verbs within that category. This can potentially be helpful if we were to see which action within each category students tended to choose.

In [18]:
def fig_frequencies(tbl, keyword):
    counter = Counter(" ".join(tbl[first_column]).split()).most_common(100)
    array = []
    for c in counter:
        array = np.append(array, c[0])

    sentence = ','.join(array)

    def extract_nouns_verbs(sentence):
        text = nltk.word_tokenize(sentence)
        pos_tagged = nltk.pos_tag(text)
        nouns_verbs = filter(lambda x:x[1]=='NN' or x[1] == 'VB',pos_tagged)
        return list(nouns_verbs)

    nouns_verbs = pd.DataFrame(extract_nouns_verbs(sentence), columns=['Word', 'Word type'])
    counted_words = pd.DataFrame(counter, columns=['Word', 'Word frequency'])

    noun_verb_frequency = nouns_verbs.merge(counted_words, on='Word', how='left').sort_values(by='Word frequency', ascending=False)

    nlp = en_core_web_sm.load()
    def all_nouns(sentence):
        doc = nlp(sentence)
        nouns = [(token.lemma_, "NN") for token in doc if token.pos_ == "NOUN"]
        return nouns

    def all_verbs(sentence):
        doc = nlp(sentence)
        verbs = [(token.lemma_, "VB") for token in doc if token.pos_ == "VERB"]
        return verbs

    nouns_verbs_spacy = all_nouns(sentence) + all_verbs(sentence)

    nouns_verbs_sp = pd.DataFrame(nouns_verbs_spacy, columns=['Word', 'Word type'])
    counted_words = pd.DataFrame(counter, columns=['Word', 'Word frequency'])

    noun_verb_frequency_sp = nouns_verbs_sp.merge(counted_words, on='Word', how='inner').sort_values(by='Word frequency', ascending=False)
    noun_verb_frequency_sp = noun_verb_frequency_sp.drop_duplicates(subset='Word', keep="last")

    fig = px.bar(noun_verb_frequency_sp, x='Word', y='Word frequency', color='Word type', title='Noun/verb frequencies for Category: ' + keyword)
    fig.update_layout(xaxis_categoryorder = 'total descending')

    fig.show()

In [19]:
fig_frequencies(scores_categories_physical, 'Physical');

/shared-libs/python3.9/py/lib/python3.9/site-packages/spacy/language.py:1895: UserWarning:

[W123] Argument disable with value [] is used instead of ['senter'] as specified in the config. Be aware that this might affect other components in your pipeline.



In [20]:
fig_frequencies(scores_categories_mental, 'Mental');

In [21]:
fig_frequencies(scores_categories_emotional, 'Emotional');

In [22]:
fig_frequencies(scores_categories_community, 'Community');

In [23]:
fig_frequencies(scores_categories_spiritual, 'Spiritual');

In [24]:
fig_frequencies(scores_categories_occupational, 'Occupational');

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=443b2d8b-ed93-43a6-bbe3-c3fe69ac80e1' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>